# Quick Start

Below is a simple demo of interaction with the environment.

In [3]:
from maro.simulator import Env
from maro.simulator.scenarios.ecr.common import Action, DecisionEvent

env = Env(scenario="ecr", topology="toy.5p_ssddd_l0.0", start_tick=0, durations=100)

is_done: bool = False
reward: int = None
decision_event: DecisionEvent = None

while not is_done:
    action: Action = None
    reward, decision_event, is_done = env.step(action)

# Environment of Empty Container Repositioning (ECR)

To initialize an environment, you need to specify the values of several parameters:
- **scenario**: The target scenario of this Env. "ecr" denotes for the Empty Container Repositioning.
- **topology**: The target topology of this Env.
   + There are some predefined topologies in MARO, that you can directly use it as in the demo.
   + Also, you can define your own topologies following the guidance in the [doc](docs/customization/new_topology.rst).
- **start_tick**: The start tick of this Env, **1 tick corresponds to 1 minute in ecr.** (TODO: to confirm)
   + In the demo above, *start_tick=0* indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of thie Env, **in the unit of tick/minute**.(TODO: to confirm)
   + In the demo above, *durations=1440* indicates a simulation length of 1 day (24h * 60min/h).

You can get all available scenarios and topologies by calling:

In [4]:
from maro.simulator.utils import get_available_envs

get_available_envs()    # TODO: specify the scenario

[{'scenario': 'citi_bike', 'topology': 'ny201912'},
 {'scenario': 'citi_bike', 'topology': 'train'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.0'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.3'},
 {'scenario': 'ecr', 'topology': '5p_ssddd_l0.6'},
 {'scenario': 'ecr', 'topology': '5p_ssddd_l0.5'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.4'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.5'},
 {'scenario': 'ecr', 'topology': '5p_ssddd_l0.2'},
 {'scenario': 'ecr', 'topology': '22p_global_trade_l0.1'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.6'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.8'},
 {'scenario': 'ecr', 'topology': '22p_global_trade_l0.3'},
 {'scenario': 'ecr', 'topology': '22p_global_trade_l0.4'},
 {'scenario': 'ecr', 'topology': '5p_ssddd_l0.8'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.2'},
 {'scenario': 'ecr', 'topology': '6p_sssbdd_l0.7'},
 {'scenario': 'ecr', 'topology': '22p_global_trade_l0.2'},
 {'scenario': 'ecr', 'topology': '5p_ssddd_

Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [26]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for ECR scenario
env = Env(scenario="ecr", topology="5p_ssddd_l0.0", start_tick=0, durations=100)

# The current tick
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The agent index list in the environment
agent_idx_list: List[int] = env.agent_idx_list
print(f"There are {len(agent_idx_list)} agents in this Env.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution
# The example of how to use the snapshot will be shown later
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary info of the environment
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary, depth=3)

The current tick: 0.
The current frame index: 0.
There are 5 agents in this Env.
There will be 100 snapshots in total.

Env Summary:
{'node_detail': {},
 'node_mapping': {'ports': {0: 'demand_port_001',
                            1: 'demand_port_002',
                            2: 'supply_port_001',
                            3: 'supply_port_002',
                            4: 'transfer_port_001'},
                  'vessels': {0: 'rt1_vessel_001',
                              1: 'rt1_vessel_002',
                              2: 'rt1_vessel_003',
                              3: 'rt2_vessel_001',
                              4: 'rt2_vessel_002',
                              5: 'rt2_vessel_003'}}}


# Interaction with the environment

Before starting interaction with the environment, we need to know DecisionEvent and Action first.

## DecisionEvent

Once the environment need the agent's response to promote the simulation, it will throw an **DecisionEvent**. In the scenario of ECR, the information of each DecisionEvent is listed as below:
- **tick**: (int) the corresponding tick
- **port_idx**: (int) the id of the port/agent that needs to respond to the environment
- **vessel_idx**: (int) the id of the vessel/operation object of the port/agnet.
- **snapshot_list**: (int) **Snapshots of the environment to input into the decision model** TODO: confirm the meaning
- **action_scope**: **Load and discharge scope for agent to generate decision**
- **early_discharge**: **Early discharge number of corresponding vessel**

## Action

Once we get a DecisionEvent from the envirionment, we should respond with an Action. Valid Action could be:

- None, which means do nothing.
- A valid Action instance, including:
   + **vessel_idx**: (int) the id of the vessel/operation object of the port/agent.
   + **port_idx**: (int) the id of the port/agent that take this action.
   + **quantity**: (int) the sign of this value denotes different meanings:
      * positive quantity means unloading empty containers from vessel to port.
      * negative quantity means loading empty containers from port to vessel.

## Generate random actions based on the DecisionEvent

The demo code in the Quick Start part has shown an interaction mode that doing nothing(responding with None action). Here we read the detailed information about the DecisionEvent and generate random actions based on it.

In [25]:
from maro.simulator import Env
from maro.simulator.scenarios.ecr.common import Action, DecisionEvent

import random

env = Env(scenario="ecr", topology="5p_ssddd_l0.0", start_tick=0, durations=100)

is_done: bool = False
reward: int = None
decision_event: DecisionEvent = None

reward, decision_event, is_done = env.step(None)

while not is_done:
    # Generate a random Action according to the action_scope in DecisionEvent
    action = Action(
        vessel_idx=decision_event.vessel_idx,
        port_idx=decision_event.port_idx,
        quantity=random.randint(-decision_event.action_scope.load, decision_event.action_scope.discharge)
    )
    # random sampling some records to show in the output
    if random.random() > 0.95:
        print(f"*************\n{decision_event}\n{action}")
    reward, decision_event, is_done = env.step(action) 

*************
DecisionEvent(tick=7, port_idx=3, vessel_idx=1, action_scope=ActionScope {load: 20000, discharge: 0 })
Action {quantity: -6886, port: 3, vessel: 1 }
*************
DecisionEvent(tick=14, port_idx=3, vessel_idx=0, action_scope=ActionScope {load: 13114, discharge: 2073 })
Action {quantity: -6744, port: 3, vessel: 0 }
*************
DecisionEvent(tick=21, port_idx=0, vessel_idx=4, action_scope=ActionScope {load: 389, discharge: 5977 })
Action {quantity: 1936, port: 0, vessel: 4 }
*************
DecisionEvent(tick=42, port_idx=3, vessel_idx=2, action_scope=ActionScope {load: 29092, discharge: 6041 })
Action {quantity: 5316, port: 3, vessel: 2 }
*************
DecisionEvent(tick=77, port_idx=3, vessel_idx=0, action_scope=ActionScope {load: 26402, discharge: 12393 })
Action {quantity: -14075, port: 3, vessel: 0 }
*************
DecisionEvent(tick=91, port_idx=0, vessel_idx=3, action_scope=ActionScope {load: 0, discharge: 6462 })
Action {quantity: 6194, port: 0, vessel: 3 }


## Get the environment observation

You can also implement other strategies or build models to take action. At this time, real-time information and historical records of the environment are very important for making good decisions. In this case, the the environment snapshot list is exactly what you need.

The information in the snapshot list is indexed by 3 dimensions:
- A frame index or a frame index list. (int or list of int) Empty indicates for all time slides till now
- A station id (list). (int of list of int) Empty indicates for all ports/agents
- An Attribute name (list). (str of list of str) You can get all available attributes in env.summary as shown before.

The return value from the snapshot list is a numpy.ndarray with shape **(frame * attribute * station, )**.

More detailed introduction to the snapshot list is [here](). # TODO: add hyper-link

In [27]:
from maro.simulator import Env
from pprint import pprint

env = Env(scenario="ecr", topology="5p_ssddd_l0.0", start_tick=0, durations=100)

pprint(env.summary)

{'node_detail': {},
 'node_mapping': {'ports': {0: 'demand_port_001',
                            1: 'demand_port_002',
                            2: 'supply_port_001',
                            3: 'supply_port_002',
                            4: 'transfer_port_001'},
                  'vessels': {0: 'rt1_vessel_001',
                              1: 'rt1_vessel_002',
                              2: 'rt1_vessel_003',
                              3: 'rt2_vessel_001',
                              4: 'rt2_vessel_002',
                              5: 'rt2_vessel_003'}}}


In [15]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for ECR scenario
env = Env(scenario="ecr", topology="5p_ssddd_l0.0", start_tick=0, durations=100)

# Run the environment to the end
_, _, is_done = env.step(None)
while not is_done:
    _, _, is_done = env.step(None)

# Get the capacity info for each ports by directly using initial frame index and attribute name
capacity = env.snapshot_list["ports"][0::"capacity"]
print(type(capacity), capacity.shape)
for i in range(len(env.agent_idx_list)):
    print(f"Port {i} capacity: {capacity[i]}")
print()
    
# Get fulfillment and shortage info simultaneously by using attribute list
attribute_list = ["fulfillment", "shortage"]
info = env.snapshot_list["ports"][::attribute_list]
print(type(info), info.shape)

# Reshape the info of fulfillment and shortage into a user-friendly shape
num_attributes = len(attribute_list)
num_frame = env.frame_index + 1
num_ports = len(env.agent_idx_list)
info = info.reshape(num_frame, num_attributes, num_ports)
print(type(info), info.shape)

# Pring and show the change of shortage in each port:
shortage_idx = 1
for port_id in env.agent_idx_list:
    print(f"Port {port_id}: {info[:, shortage_idx, port_id]}")

<class 'numpy.ndarray'> (5,)
Port 0 capacity: 100000.0
Port 1 capacity: 100000.0
Port 2 capacity: 100000.0
Port 3 capacity: 100000.0
Port 4 capacity: 100000.0

<class 'numpy.ndarray'> (1000,)
<class 'numpy.ndarray'> (100, 2, 5)
Port 0: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500.]
Port 1: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 500. 500.
 500. 500. 500. 500. 500. 500. 500